# Yoga: Pet Detective

## Standard Deep Atlas Exercise Set Up

- [ ] Ensure you are using the coursework Pipenv environment and kernel ([instructions](../SETUP.md))
- [ ] Apply the standard Deep Atlas environment setup process by running this cell:

In [1]:
import sys, os
sys.path.insert(0, os.path.join('..', 'includes'))
import deep_atlas

### 🚦 Checkpoint: Start

- [ ] Run this cell to record your start time:

In [2]:
deep_atlas.log_start_time()

Started at: 2025-05-13T10:38:27.920696
🚀 Success! Get started...


In [3]:
!pipenv install torch==2.1.2 torchvision==0.16.2 torchinfo==1.8.0

Courtesy Notice:
Pipenv found itself running within a virtual environment,  so it will 
automatically use that environment, instead of  creating its own for any 
project. You can set
PIPENV_IGNORE_VIRTUALENVS=1 to force pipenv to ignore that environment and 
create  its own instead.
You can set PIPENV_VERBOSITY=-1 to suppress this warning.
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.
Installing torch==2.1.2...
✔ Installation Succeeded
Installing torchvision==0.16.2...
✔ Installation Succeeded
Installing torchinfo==1.8.0...
✔ Installation Succeeded
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.
Installing dependencies from Pipfile.lock (f77f4c)...
All dependencies are now up-to-date!
Upgrading torch==2.1.2, torchvision==0.16.2, torchinfo==1.8.0 in  dependencies.
Building requirements...
Resolving dependencies....
✔ Success! Locking

In [9]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from torchvision.models import resnet18, ResNet18_Weights, AlexNet, AlexNet_Weights , MobileNetV2, MobileNet_V2_Weights

In [15]:
import os
import shutil
from pathlib import Path

def organize_images(source_dir):
    # Create the main images directory if it doesn't exist
    images_dir = Path("./images")
    images_dir.mkdir(exist_ok=True)
    
    # Get all image files from the source directory
    image_extensions = ('.jpg', '.jpeg', '.png', '.gif', '.bmp')
    image_files = [f for f in os.listdir(source_dir) if f.lower().endswith(image_extensions)]
    
    # Process each image
    for image_file in image_files:
        # Get the name (assuming format is "name_number.jpg" or similar)
        # You might need to adjust this based on your actual filename format
        name = image_file.split('_')[0]  # Gets the first part before underscore
        
        # Create subdirectory for this name if it doesn't exist
        subdir = images_dir / name
        subdir.mkdir(exist_ok=True)
        
        # Copy the image to its new location
        source_path = Path(source_dir) / image_file
        dest_path = subdir / image_file
        shutil.copy2(source_path, dest_path)
        print(f"Copied {image_file} to {name}/")

if __name__ == "__main__":
    # Replace this with the path to your source directory containing the images
    source_directory = input("Enter the path to your source directory containing the images: ")
    
    if os.path.exists(source_directory):
        organize_images(source_directory)
        print("\nOrganization complete! Check the 'images' folder for the results.")
    else:
        print("Error: The specified directory does not exist.")

Copied Egyptian_Mau_167.jpg to Egyptian/
Copied pug_52.jpg to pug/
Copied basset_hound_112.jpg to basset/
Copied Siamese_193.jpg to Siamese/
Copied shiba_inu_122.jpg to shiba/
Copied Siamese_53.jpg to Siamese/
Copied Birman_167.jpg to Birman/
Copied leonberger_6.jpg to leonberger/
Copied Siamese_47.jpg to Siamese/
Copied shiba_inu_136.jpg to shiba/
Copied saint_bernard_139.jpg to saint/
Copied Birman_173.jpg to Birman/
Copied Abyssinian_225.jpg to Abyssinian/
Copied basset_hound_106.jpg to basset/
Copied miniature_pinscher_200.jpg to miniature/
Copied Siamese_187.jpg to Siamese/
Copied wheaten_terrier_49.jpg to wheaten/
Copied Egyptian_Mau_173.jpg to Egyptian/
Copied pug_46.jpg to pug/
Copied shiba_inu_43.jpg to shiba/
Copied wheaten_terrier_61.jpg to wheaten/
Copied saint_bernard_111.jpg to saint/
Copied scottish_terrier_132.jpg to scottish/
Copied scottish_terrier_126.jpg to scottish/
Copied saint_bernard_105.jpg to saint/
Copied saint_bernard_2.jpg to saint/
Copied pomeranian_189.jp

In [24]:
data = datasets.ImageFolder(
    "./images", transform=transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                           std=[0.229, 0.224, 0.225])
    ])
)

In [25]:
# First split into train/test
train_paths, test_paths, train_labels, test_labels = train_test_split(
    image_paths, labels, test_size=0.2, random_state=42
)
# Then split train into train/val
train_paths, val_paths, train_labels, val_labels = train_test_split(
    train_paths, train_labels, test_size=0.25, random_state=42
)

NameError: name 'train_test_split' is not defined

In [17]:
# 70% of data is used for training
train_size = int(0.7 * len(data))
# 15% of data is used for validation
val_size = int(0.15 * len(data))
# The remaining 15% is used for testing
test_size = len(data) - train_size - val_size

train_data, val_data, test_data = random_split(
    data, [train_size, val_size, test_size]
)

In [18]:
# Create dataloaders
batch_size = 64
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=batch_size)

In [19]:
model = resnet18(weights=ResNet18_Weights.DEFAULT, progress=False)

In [20]:
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 37)

In [21]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
device = torch.device(
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")
model = model.to(device)

Using mps device


In [22]:
def calculate_accuracy(loader):
    correct = 0
    total = 0
    # Set model to evaluation mode
    model.eval()
    # Disable gradient calculation (unnecessary for inference)
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

In [23]:
val_accuracy = calculate_accuracy(val_loader)
print(f"Validation accuracy before fine-tuning: {val_accuracy}%")

Validation accuracy before fine-tuning: 2.436823104693141%


## Goals:

- Develop a deep learning model that can classify images of pets into specific categories from a provided dataset.
- Use transfer learning to fine-tune a pre-trained model on a new, custom dataset, leveraging an existing architecture and corresponding weights to achieve accuracy with limited data.

## Dataset:

Use the [Oxford-IIIT Pet Dataset](https://www.robots.ox.ac.uk/~vgg/data/pets/), which contains images of 37 pet breeds (cats and dogs) with annotations. This dataset is balanced and provides a moderate challenge.

## Base models: 

- [AlexNet](https://pytorch.org/vision/stable/models/alexnet): A historical model developed to win the 2012 ImageNet Large Scale Visual Recognition Challenge. It pioneered the deep architecture of five convolutional layers followed by three fully connected layers (it was also the first model to implement Rectified Linear Units (ReLUs) for the non-linear layers, which allowed it to train much faster than networks with sigmoid activations).
    - Approachable architecture, especially if you aim to partially _freeze_ the parameters in particular layers. 
- [ResNet](https://pytorch.org/vision/stable/models/resnet.html): A popular choice for image classification tasks due to its performance and efficiency. It strikes a good balance between computational demand and learning capability.
- [MobileNet](https://pytorch.org/vision/stable/models/mobilenetv2): A lighter-weight option designed for mobile and resource-constrained environments, making it suitable for running on laptops.

## Extra Credit: 

- [ ] Export and wrap your model in a web-app (e.g. a game that allows users to compete with the bot in multiple-choice pet-guessing game).
- [ ] Fine tune an image segmentation model using the data included with the Oxford-IIIT dataset.


## Resources

- [PyTorch: Transfer Learning](https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html)

### 🚦 Checkpoint: Stop

- [ ] Uncomment this code
- [ ] Complete the feedback form
- [ ] Run the cell to log your responses and record your stop time:

In [ ]:
# deep_atlas.log_feedback(
#     {
#         # How long were you actively focused on this section? (HH:MM)
#         "active_time": FILL_THIS_IN,
#         # Did you feel finished with this section (Yes/No):
#         "finished": FILL_THIS_IN,
#         # How much did you enjoy this section? (1–5)
#         "enjoyment": FILL_THIS_IN,
#         # How useful was this section? (1–5)
#         "usefulness": FILL_THIS_IN,
#         # Did you skip any steps?
#         "skipped_steps": [FILL_THIS_IN],
#         # Any obvious opportunities for improvement?
#         "suggestions": [FILL_THIS_IN],
#     }
# )
# deep_atlas.log_stop_time()

## You did it!